In [1]:
import pandas as pd

import sys
sys.path.append('../pycore/')
from utils import metabolites_dict_from_reaction_equation_RBA

In [2]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_eqn = pd.read_excel('../scRBA_build_GAMS_model/model/RBA_stoichiometry.xlsx')
df_eqn.index = df_eqn.id.to_list()

# Get proteins
met_list_raw = []
for i in df_eqn.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_eqn.reaction[i])
    met_list_raw += met_dict.keys()

met_list = sorted(list(set(met_list_raw)))
met_list = [i for i in met_list if i != '']
met_list = [i for i in met_list if i.split('-', maxsplit=1)[0] in ['PRO']]

In [3]:
df_data = pd.read_excel('./Bjorkeroth_Nielsen_2020_data.xlsx')
df_data.index = df_data['id'].to_list()
df_data = df_data[df_data['conc (g/gDW)'] > 0]

In [4]:
data = []; pro_data = []; pro_nodata = []
for met in met_list:
    _,sid = met.split('-', maxsplit=1)
    if sid in df_data.index:
        pro_data.append("'PROIN-" + sid + "'")
        data.append("'PROIN-" + sid + "' " + str(df_data.loc[sid, 'vtrans (mmol/gDW/h)']))
    else:
        pro_nodata.append("'PROIN-" + sid + "'")
        
data = ['/'] + data + ['/']
pro_data = ['/'] + pro_data + ['/']
pro_nodata = ['/'] + pro_nodata + ['/']

In [5]:
with open('./enz_from_proteome/proteome_data.txt', 'w') as f:
    f.write('\n'.join(data))

In [6]:
with open('./enz_from_proteome/rxns_pro_data.txt', 'w') as f:
    f.write('\n'.join(pro_data))

In [7]:
with open('./enz_from_proteome/rxns_pro_nodata.txt', 'w') as f:
    f.write('\n'.join(pro_nodata))